In [ ]:
from os.path import join as pjoin
from os.path import exists as exist
from os import listdir as lsdir
from os import makedirs
from pathlib import Path
from obspy.geodetics.base import gps2dist_azimuth
from collections import OrderedDict


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
import matplotlib.dates as mpldates
from matplotlib import rcParams
from matplotlib import colors
from matplotlib import cm
import pygmt

In [ ]:
data = pd.read_csv('Output/Others/Data_ReguibatClust.csv')

In [ ]:
datamagnitude = data['Magnitude']
datatime = pd.to_datetime(data['Time'])

data['year'] = datatime.dt.year
datayear = data['year']
data.insert(1, "Year",datayear, True)
data.sort_values(by=['Year'],inplace=True, ascending=True)
data

In [ ]:
data_2 = data.loc[(data["MagType"] ==  "mb")]
data_2

In [ ]:
datatimeM0 = pd.to_datetime(data_2['Time'])
datatimeM0 = datatimeM0.dt.year
Mbs = data_2["Magnitude"].to_numpy()
#Magnitude conversion
Mws = 0.85 * Mbs + 1.03
exponent = (3*Mws + 10.7)/2
M0s = 10**(exponent)
csumMOs = np.cumsum(M0s)
csumMOs


In [ ]:
countyear = datayear.value_counts()
countyear.sort_index(inplace=True,ascending=True)
countyear=countyear.to_frame().reset_index()
countyear.rename(columns = {'index':'Year', 'year':'Eq_counts'}, inplace = True)

countyear

In [ ]:
topo_data = '@earth_relief_30s'

In [ ]:
plt.style.use('default')
plt.rcParams["figure.figsize"] = (40,10)

fig, ax = plt.subplots()
# This is  the colormap I'd like to use.
cm = plt.get_cmap('hot',20)
cmaplist = [cm(i) for i in range(cm.N)]
# force the first color entry to be grey
cmaplist[0] = "black"
cmaplist[1] = "black"
cmaplist[2] = "grey"
cmaplist[3] = "grey"


cmaplist[4] = "yellow"
cmaplist[5] = "yellow"
cmaplist[6] = "yellow"


cmaplist[7] = "orange"
cmaplist[8] = "orange"
cmaplist[9] = "orange"

cmaplist[10] = "red"
cmaplist[11] = "red"
cmaplist[12] = "red"
cmaplist[13] = "red"

cmaplist[14] = "salmon"
cmaplist[15] = "salmon"

# # create the new map
cm = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cm.N)

datayear[-1]=2022
# Plot histogram.
n, bins, patches = plt.hist(datayear,21,edgecolor= 'black')
bin_centers = 0.5 * (bins[:-1] + bins[1:])

# scale values to interval [0,1]
col = bin_centers - min(bin_centers)
col /= max(col)


for c, p in zip(col, patches):
    plt.setp(p, 'facecolor', cm(c))
    

plt.xlim(2000,2023);
plt.xticks([2001,2003,2005,2008,2011,2015,2017,2021,2022]);
plt.yticks([0,10,20,30,40]);
plt.tick_params(axis="y", labelsize=35,direction="out",length=12,width=3,labelcolor = "red")
plt.tick_params(axis="x", labelsize=35,direction="out",length=12,width=3,labelcolor = "black")
plt.ylabel("Earthquake Counts", fontsize=40,color = "red",fontweight="bold")
plt.xlabel("Years", fontsize=40,fontweight="bold")

ax = plt.gca()
ax2 = ax.twinx()
ax2.plot(datatimeM0,csumMOs,color="black",linewidth=3)
ax2.tick_params(axis="both", labelsize=35,direction="out",length=12,width=3,color="black")
ax2.set_ylabel("Cumulative Moment\n(dyne*cm)",fontsize=40,fontweight="bold")
for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(3)
        
output ="Output/Figures/"
plt.savefig(pjoin(output,"Figure_S2a.pdf"),dpi=600,)        
plt.show()

In [ ]:

plt.rcParams["figure.figsize"] = (40,40)
fig = pygmt.Figure()
pygmt.makecpt(
    cmap='gray',
    series='-30000/3000/500',
    continuous=True
)

fig.grdimage(
        grid=topo_data,
        region=[-14, -9, 21, 25],
        projection="M8c",
        shading=True,
        frame="a",
)

fig.coast(region=[-14, -9, 21, 25],projection="M8c",#land="lightgray", water="white",
          borders="1/0.2p,dimgray",
          shorelines="1/0.5p",resolution="h")

fig.text(projection="M8c",text="Mauritania", x= -10, y=22,font="15p,dimgray")

yupper = [2017,2014,2010,2007,2004,2021,2002]
ylower = [2015,2011,2008,2005,2003,2020,2001]
colorlist = ["salmon", "red", "orange","yellow","grey","white","black"]
for zipped in zip(colorlist,yupper,ylower):
    fill = zipped[0]
    yup = zipped[1]
    yl = zipped[2]
    datamap = data.loc[(data['Year'] >= yl) & (data['Year'] <= yup)]
    size=datamap["Magnitude"]*0.1
    longitude = datamap["Longitude"]
    latitude = datamap["Latitude"]
    # plot data points as circles (style="c"), with different sizes
    fig.plot(
        x=longitude, 
        y=latitude,
        pen="0.3p,black",
        style="cc",
        size=size,
        color=fill,
    )
     
fig.plot(
    projection="M8c",
    x=-12.45, 
    y=22.65,
    pen="0.3p,black",
    style="g0.3c",
    color="blue",
)  

plot_longitude = -12.45
plot_latitude = 22.65 +1

fig.plot(projection="M8c",x=[-12.45,plot_longitude],y=[22.65,plot_latitude],pen="2p,blue")

fig.plot(
    projection="M8c",
    x=-12.45, 
    y=22.65,
    pen="0.3p,black",
    style="g0.3c",
    color="blue",
)                

               
fig.text(
    projection="M8c",
    x=plot_longitude, 
    y=plot_latitude+0.15,text="Kedia d'idjill Mine",
    font="15p,blue",
)                
                
    
output ="Output/Figures/Figure_S2b.pdf"   
fig.savefig(output, transparent=False, crop=True, anti_alias=True, show=True)

fig.show()

In [ ]:

fig, ax = plt.subplots(figsize=(30, 5))
fig.subplots_adjust(bottom=0.5)

cmap = mpl.colors.ListedColormap([ 'black',"grey",'yellow','orange','orange','red','salmon','white','white','white'])
cmap.set_over('0.25')
cmap.set_under('0.75')




bounds = [2001,2003,2005,2008,2011,2015,2017,2021,2022]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
cb2 = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
                                norm=norm,
                                boundaries=[0] + bounds + [13],
                                extend='both',
                                ticks=bounds,
                                spacing='proportional',
                                orientation='horizontal')
cb2.set_label('Discrete intervals, some other units')

ax.tick_params(axis="both",labelsize=40,direction="out",length=12,width=3)  

output ="Output/Figures/Figure_bar_earthquakes.pdf"   
fig.savefig(output, transparent=False, crop=True, anti_alias=True, show=True)

fig.show()